<img src="img/Logo_OpenClassrooms.png" width=100 />

# OpenClassrooms Projet 5: Segmentez des clients d'un site e-commerce
# Analyse Exploratoire

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Votre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

Vous devrez fournir à l’équipe marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

### Les données
Pour cette mission, Olist vous fournit une base de données anonymisée comportant des informations sur l’historique de commandes, les produits achetés, les commentaires de satisfaction, et la localisation des clients depuis janvier 2017.

### Votre mission
Votre mission est d’aider les équipes d’Olist à comprendre les différents types d'utilisateurs. Vous utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires. Ces catégories pourront être utilisées par l’équipe marketing pour mieux communiquer.

Pour des raisons de confidentialité, Olist ne fournit pas beaucoup de données, à vous de fouiller dans celles dont vous disposez et de créer les meilleures features pour les exploiter.

Enfin, votre client, Olist, a spécifié sa demande ainsi :

- La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
- Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
- Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.


# Sommaire
### [Bibliothèques](#1_bibli)
### [Fonctions](#1_funcs)
### [Données](#1_donnees)
### [Création de la base](#1_creation_base)


<a id='1_bibli'></a>
# Import de bibliothèques 📚

In [1]:
import pandas as pd
pd.set_option("mode.chained_assignment", None)
import numpy as np

<a id='1_funcs'></a>

# Fonctions ⚙️

<a id='get_orders'></a>

In [2]:
def get_orders(client_unique_id):
    """ A partir d'un id client unique la fonction retourne la liste des commandes correspondantes,
    La table des commandes doit au préalable être filtrée en fonction de la date virtuelle de l'étude
    Args:
        client_unique_id
    """
    
    # Avec l'id unique du client on récupère les id clients qui sont liés à chaque commande
    ids_client = customers[customers['customer_unique_id'] == client_unique_id].customer_id.values
    # On peut ensuite récupérer les id des commandes correspondant à ces ids clients
    orders_client = orders[orders.customer_id.isin(ids_client)]
    # On récupère juste les ids
    ids_orders_client = orders_client.index.values
    return ids_orders_client

[Retour au code](#get_orders_back)

<a id='get_nb_days'></a>

In [3]:
def get_nb_days(row, virtual_date, orders):
    """
    Cette fonction calcule le nombre de jours écoulés depuis la première et la dernière 
    commande en prenant pour référence la date virtuelle de l'étude.
    Args :
        row: Pandas DataFrame row (Series) avec un attribut "orders_ids"
        Timestamp 
        Pandas DataFrame : avec les données des commandes
    Returns :
        Tuple
    """
    orders_ids = row.orders_ids
    
    date_first_order = orders.loc[orders_ids[0], 'order_purchase_timestamp']
    date_last_order = orders.loc[orders_ids[-1], 'order_purchase_timestamp']
    
    since_first = (virtual_date - date_first_order).days
    since_last = (virtual_date - date_last_order).days
    return since_first, since_last

[Retour au code](#tps_last_order)

<a id='get_frequency'></a>

In [4]:
def get_frequency(row):
    """
    Calcul du nombre de commandes par mois entre la première commande
    et la date virtuelle.
    """
    return row.nb_orders / row.days_since_first_order * 365.25 / 12

[Retour au code](#frequence)

<a id='get_sum_orders'></a>

In [5]:
def get_sum_orders(row):
    """
    Calcule la somme des montants des commandes
    """
    return order_items[order_items.index.isin(row.orders_ids)].price.sum()    

[Retour au code](#montant)

<a id='get_most_frequent_categ'></a>

In [6]:
def get_most_frequent_categ(row):
    """
    Retourne la catégorie qui revient le plus souvent dans les achats. 
    En cas d'égalité on prend la première donnée par value_counts()
    """
    
    orders_ids = row.orders_ids
    products_ids = order_items[order_items.index.isin(row.orders_ids)].product_id
    try:
        return products.loc[products_ids.value_counts().index[0]].product_category_name_english 
    except:
        if len(products_ids) > 1:
            return products.loc[products_ids.value_counts().index[1]].product_category_name_english
        
        else :
            return np.nan

[Retour au code](#categorie)

<a id='get_nb_reviews_and_avg'></a>

In [7]:
def get_nb_reviews_and_avg(row):
    """
    Recupère le nombre de reviews et la note moyenne
    """
    orders_ids = row.orders_ids
    client_reviews = reviews[reviews.order_id.isin(orders_ids)]
    if len(client_reviews) > 0:
        return len(client_reviews), client_reviews.review_score.mean()
    else:
        return 0, np.nan

[Retour au code](#reviews)

<a id='get_favourite_payment_method'></a>

In [8]:
def get_favourite_payment_method(row):
    """
    Retourne le type de paiement préféré du client
    """
    orders_ids = row.orders_ids
    payment_methods = payments[payments.order_id.isin(orders_ids)].payment_type.value_counts()
    return payment_methods.index[0]

[Retour au code](#mode_paiement_prefere)

<a id='1_donnees'></a>

# Données 🎁

<img src="img/structure.png" width=500 />

In [9]:
orders = pd.read_csv('data/olist_orders_dataset.csv', index_col='order_id',parse_dates=['order_purchase_timestamp', 
                                                                                        'order_approved_at',	
                                                                                        'order_delivered_carrier_date',	
                                                                                        'order_delivered_customer_date',
                                                                                        'order_estimated_delivery_date'])
order_items = pd.read_csv('data/olist_order_items_dataset.csv',  index_col='order_id')
products = pd.read_csv('data/olist_products_dataset.csv', index_col='product_id', na_values=np.nan)
customers = pd.read_csv('data/olist_customers_dataset.csv')
reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
geolocs = pd.read_csv('data/olist_geolocation_dataset.csv')
payments = pd.read_csv('data/olist_order_payments_dataset.csv')
products_categ_traduction = pd.read_csv('data/product_category_name_translation.csv')

Application de la traduction des catégories de produits:

In [10]:
products = products.merge(products_categ_traduction, how='left').set_axis(products.index)

<a id='1_creation_base'></a>

# Création d'une base répondant au besoin ✅
La base doit avoir pour index les ids des clients avec des variables en relation avec son comportement.

Variables retenues:
- [Nombre de commandes](#nb_commandes)
- [Temps depuis la première et la dernière commande](#tps_last_order)
- [Fréquence achats](#frequence)
- [Montants dépensés](#montant)
- [Catégorie de produit la plus fréquente](#categorie)
- [Nombre d'avis postés et note moyenne](#reviews)
- [Mode de paiement préféré](#mode_paiement_prefere)
- [Temps de livraison et retards]

### Important:
Pour le bon fonctionnement du projet, les fonctions développées pour la constitution de la base prendront en paramètre la date à laquelle on veut se fixer virtuellement. En effet on cherchera plus tard dans ce projet à déterminer la fréquence de mise à jour de la base d'entrainement du modèle.
On prendra toujours dans ce notebook la dernière date de la base de donnée.

In [11]:
virtual_date = orders.order_purchase_timestamp.max()

In [12]:
orders.order_purchase_timestamp.max()

Timestamp('2018-10-17 17:30:18')

<a id='nb_commandes'></a>

## Nombre de commandes par client 🥰

Cette variable est la plus importante car va nous permettre de connaitre la taille de notre dataset. En effet, difficile de classer des clients n'ayant fait qu'une commande

Regardons comment fonctionnent les id des clients

In [13]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [14]:
len(customers.customer_unique_id.unique())

96096

In [15]:
len(customers.customer_id.unique())

99441

In [16]:
orders.columns

Index(['customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

On filtre ici la date virtuelle pour garder un trace mais cette commande est neutre

In [17]:
orders = orders[orders.order_purchase_timestamp <= virtual_date]

In [18]:
orders = orders.merge(customers[['customer_id', 'customer_unique_id']], how='left').set_axis(orders.index)

On trie par date pour conserver un ordre des commandes logique

In [19]:
orders.sort_values('order_purchase_timestamp', inplace=True)

In [20]:
(orders.groupby('customer_unique_id').count().customer_id > 1).value_counts()

False    93099
True      2997
Name: customer_id, dtype: int64

### Il n'y a dans la base clients que 2997 clients ayant fait plus d'un commande

In [21]:
data = pd.DataFrame(orders.groupby('customer_unique_id').count().customer_id)

In [22]:
data.columns = ['nb_orders']

Nous avons besoin de clients ayant plus d'une commande pour commencer à comprendre leur comportement. Cela réduit considérablement la taille de la base

In [23]:
data = data[data.nb_orders > 1]

<a id='get_orders_back'></a>
### Pour les autres variables nous avons besoin des ids des commandes correspondant à chaque client

Voir la [fonction](#get_orders)

In [24]:
get_orders.__doc__

" A partir d'un id client unique la fonction retourne la liste des commandes correspondantes,\n    La table des commandes doit au préalable être filtrée en fonction de la date virtuelle de l'étude\n    Args:\n        client_unique_id\n    "

In [25]:
data['orders_ids'] = data.index.map(get_orders)

In [26]:
data.head()

,nb_orders,orders_ids
customer_unique_id,,
00172711b30d52eea8b313a7f2cced02,2,"[bb874c45df1a3c97842d52f31efee99a, c306eca42d3..."
004288347e5e88a27ded2bb23747066c,2,"[a61d617fbe5bd006e40d3a0988fc844b, 08204559beb..."
004b45ec5c64187465168251cd1c9c2f,2,"[90ae229a4addcfead792e2564554f09c, 9392c5e7288..."
0058f300f57d7b93c477a131a59b36c3,2,"[2cfc79d9582e9135c0a9b61fa60e6b21, 81a93b2fa39..."
00a39521eb40f7012db50455bf083460,2,"[7d32c87acba91ed87ebd98310fe1c54d, cea3e6c11eb..."


<a id='tps_last_order'></a>
## Temps depuis la première et la dernière commande ⏲️
Pour représenter l'ancienneté du client et la fraicheur du client. On aura besoin ici de la date virtuelle

Voir la [fonction](#get_nb_days)

In [27]:
data[['days_since_first_order', 'days_since_last_order']] = data.apply(get_nb_days, virtual_date=virtual_date, 
                                                                       orders=orders, axis=1, result_type='expand')

In [28]:
data[['days_since_first_order', 'days_since_last_order']].describe()

,days_since_first_order,days_since_last_order
count,2997.000000,2997.000000
mean,355.512846,268.206874
std,148.227400,145.352456
min,49.000000,0.000000
25%,239.000000,152.000000
50%,350.000000,248.000000
75%,474.000000,366.000000
max,744.000000,740.000000


<a id='frequence'></a>
## Fréquence des commandes
Pour représenter la régularité du client. On se basera aussi sur la date virtuelle

Voir la [fonction](#get_frequency)

La fréquence est exprimée par mois

In [29]:
data['frequency'] = data.apply(get_frequency, axis=1)

In [30]:
data['frequency'].describe()

count    2997.000000
mean        0.234270
std         0.167588
min         0.081931
25%         0.133206
50%         0.183912
75%         0.262392
max         1.863520
Name: frequency, dtype: float64

<a id='montant'></a>
## Montant total des achats 💰

Voir la [fonction](#get_sum_orders)

In [31]:
data['sum_orders'] = data.apply(get_sum_orders, axis=1)

In [32]:
data['sum_orders'].describe()

count    2997.000000
mean      259.867191
std       308.511754
min         0.000000
25%       109.900000
50%       179.890000
75%       303.160000
max      7388.000000
Name: sum_orders, dtype: float64

<a id='categorie'></a>
## Catégogie la plus fréquente 🗂️
On essaie ici de détecter le type de produit intéressant le plus le client en prenant la catégoie qu'il a le plus acheté

Voir la [fonction](#get_most_frequent_categ)

In [33]:
data['favourite_category'] = data.apply(get_most_frequent_categ, axis=1)

In [34]:
data['favourite_category'].isnull().value_counts()

False    2956
True       41
Name: favourite_category, dtype: int64

Certain produits ne sont pas reliés à une catégorie. 
### QUE FAIRE DES NaN??? : remplir avec la plus fréquente

Cardinalité A REDUIRE : 

In [35]:
len(data['favourite_category'].unique())

66

In [60]:
len(products.product_category_name.unique())

74

In [36]:
data['favourite_category'].value_counts()

bed_bath_table               419
furniture_decor              306
sports_leisure               293
health_beauty                238
computers_accessories        200
                            ... 
music                          1
la_cuisine                     1
tablets_printing_image         1
party_supplies                 1
fashion_childrens_clothes      1
Name: favourite_category, Length: 65, dtype: int64

<a id='reviews'></a>
## Nombre d'avis postés et note moyenne⭐
Implication et satisfaction du client sur le site

Voir la [fonction](#get_nb_reviews_and_avg)

In [37]:
data[['nb_reviews', 'average_review_score']] = data.apply(get_nb_reviews_and_avg, axis=1, result_type='expand')

In [38]:
data.nb_reviews.value_counts()

2.0     2424
3.0      405
4.0      125
5.0       23
6.0       13
7.0        5
9.0        1
17.0       1
Name: nb_reviews, dtype: int64

In [39]:
data.nb_orders.value_counts()

2     2745
3      203
4       30
5        8
6        6
7        3
9        1
17       1
Name: nb_orders, dtype: int64

In [62]:
reviews.review_score.value_counts(normalize=True)

5    0.57420
4    0.19200
1    0.11858
3    0.08287
2    0.03235
Name: review_score, dtype: float64

### C'est louche

In [40]:
data.average_review_score.describe()

count    2997.000000
mean        4.098780
std         1.150088
min         1.000000
25%         3.500000
50%         4.500000
75%         5.000000
max         5.000000
Name: average_review_score, dtype: float64

In [41]:
reviews.review_score.isnull().value_counts()

False    100000
Name: review_score, dtype: int64

On note ici que dans cette base, tous les clients écrivent des reviews et donnent des notes

<a id='mode_paiement_prefere'></a>
## Mode de paiement préféré 💳
Relation du client avec l'argent

Voir la [fonction](#get_favourite_payment_method)

In [42]:
payments.payment_type.value_counts()

credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: payment_type, dtype: int64

In [43]:
payments.payment_type.isnull().value_counts()

False    103886
Name: payment_type, dtype: int64

In [44]:
data['favourite_payment_type'] = data.apply(get_favourite_payment_method, axis=1).replace('boleto', 'cash')

In [45]:
data['favourite_payment_type'].value_counts()

credit_card    2168
cash            646
voucher         119
debit_card       63
not_defined       1
Name: favourite_payment_type, dtype: int64

<a id='tps_livraison'></a>
## Temps et retards de livraison 🚚
Expérience logistique du client. pour cette partie nous aurons besoin de créer des variables dans la table "orders"

Nous regarderons le temps moyen de chaque livraison du client et les temps de retard/avance s'il y en a eu

Voir la [fonction](#get_favourite_payment_method)

In [46]:
orders['delivery_time'] = orders.order_delivered_customer_date - orders.order_approved_at

In [47]:
orders['delay'] = orders.order_delivered_customer_date - orders.order_estimated_delivery_date

In [48]:
orders['was_delayed'] = orders.delay.map(lambda x : x.days > 1)

In [49]:
orders.head()

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,delivery_time,delay,was_delayed
order_id,,,,,,,,,,,
2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19,2016-10-07 13:18:03,2016-10-18 13:14:51,NaT,2016-10-20,b7d76e111c89f7ebf14761390f0f7d17,NaT,NaT,False
e5fa5a7210941f7d56d0208e4e071d35,683c54fc24d40ee9f8a6fc179fd9856c,canceled,2016-09-05 00:15:34,2016-10-07 13:17:15,NaT,NaT,2016-10-28,4854e9b3feff728c13ee5fc7d1547e92,NaT,NaT,False
809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-09-13 15:24:19,2016-10-07 13:16:46,NaT,NaT,2016-09-30,009b0127b727ab0ba422f6d9604487c7,NaT,NaT,False
bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04,830d5b7aaa3b6f1e9ad63703bec97d23,54 days 19:31:00,36 days 07:47:38,True
71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-02 22:07:52,2016-10-06 15:50:56,NaT,NaT,2016-10-25,0eb1ee9dba87f5b36b4613a65074337c,NaT,NaT,False


In [50]:
orders[orders['delivery_time'].isnull() & (orders.order_status == 'delivered')]

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,delivery_time,delay,was_delayed
order_id,,,,,,,,,,,
c1d4211b3dae76144deccd6c74144a88,684cb238dc5b5d6366244e0e0776b450,delivered,2017-01-19 12:48:08,NaT,2017-01-25 14:56:50,2017-01-30 18:16:01,2017-03-01,6ff8b0d7b35d5c945633b8d60165691b,NaT,-30 days +18:16:01,False
7002a78c79c519ac54022d4f8a65e6e8,d5de688c321096d15508faae67a27051,delivered,2017-01-19 22:26:59,NaT,2017-01-27 11:08:05,2017-02-06 14:22:19,2017-03-16,d49f3dae6bad25d05160fc17aca5942d,NaT,-38 days +14:22:19,False
12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,NaT,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20,c8822fce1d0bfa7ddf0da24fff947172,NaT,-18 days +11:09:19,False
3c0b8706b065f9919d0505d3b3343881,d85919cb3c0529589c6fa617f5f43281,delivered,2017-02-17 15:53:27,NaT,2017-02-22 11:31:30,2017-03-03 11:47:47,2017-03-23,c094ac95fcd52f821809ec232a7a6956,NaT,-20 days +11:47:47,False
2eecb0d85f281280f79fa00f9cec1a95,a3d3c38e58b9d2dfb9207cab690b6310,delivered,2017-02-17 17:21:55,NaT,2017-02-22 11:42:51,2017-03-03 12:16:03,2017-03-20,5a4fa4919cbf2b049e72be460a380e5b,NaT,-17 days +12:16:03,False
d77031d6a3c8a52f019764e68f211c69,0bf35cac6cc7327065da879e2d90fae8,delivered,2017-02-18 11:04:19,NaT,2017-02-23 07:23:36,2017-03-02 16:15:23,2017-03-22,c4c0011e639bdbcf26059ddc38bd3c18,NaT,-20 days +16:15:23,False
8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,NaT,2017-02-23 09:01:52,2017-03-02 10:05:06,2017-03-21,91efb7fcabc17925099dced52435837f,NaT,-19 days +10:05:06,False
7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:47,NaT,2017-02-22 16:25:25,2017-03-01 08:07:38,2017-03-17,e1f01a1bd6485e58ad3c769a5427d8a8,NaT,-16 days +08:07:38,False
e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17,8a9a08c7ca8900a200d83cf838a07e0b,NaT,-16 days +13:25:33,False


In [51]:
def get_delivery_time_and_delays(row):
    client_orders = orders[orders.index.isin(row.orders_ids)]
    client_orders = client_orders[client_orders.order_status == 'delivered']
    
    if client_orders.shape[0] > 0:
        was_delayed_counts = client_orders.was_delayed.value_counts(normalize=True)
        if True in was_delayed_counts.index:
            delay_rate = was_delayed_counts.loc[True]
        else:
            delay_rate = 0
        return client_orders.delivery_time.mean(), delay_rate
    else:
        np.nan, np.nan

In [52]:
data[['average_delivery_time', 'delay_rate']] = data.apply(get_delivery_time_and_delays, axis=1, result_type='expand')

In [63]:
data[data.average_delivery_time.isnull()]

,nb_orders,orders_ids,days_since_first_order,days_since_last_order,frequency,sum_orders,favourite_category,nb_reviews,average_review_score,favourite_payment_type,average_delivery_time,delay_rate
customer_unique_id,,,,,,,,,,,,
0af334fc660bfc9c75109752cc8271d7,2,"[3d3742a96f24a8fe4e2e57628807e476, e1e8e3bca90...",549,549,0.110883,430.00,sports_leisure,3.0,1.0,credit_card,NaT,NaN
1a674d0bad43d5b5c387fce838e4c428,2,"[a3ef7e4271c9088cf2d3de8305ef2355, 19bbda1f882...",347,347,0.175432,0.00,NaN,2.0,1.0,credit_card,NaT,NaN
22c8d7801061f27eff57f37ac4ef10e1,2,"[504ec2b6f69e898c557955750272d563, e5e34f7a2ae...",162,162,0.375772,76.98,fashion_bags_accessories,3.0,5.0,credit_card,NaT,NaN
391d6062da3dd65b4de4524f28c478de,2,"[4c8b9947280829d0a8b7e81cc249b875, 27667a063cb...",69,50,0.882246,0.00,NaN,2.0,1.0,voucher,NaT,NaN
3eb7932f075743f0e50b0d9966707e6f,2,"[858f8fbff9d717450f931fc6e9d15029, 1aba68837cf...",372,359,0.163642,0.00,NaN,2.0,4.0,cash,NaT,NaN
5ace05247b6926d3e595ac4de6620b1d,2,"[ec24ea9acb22b2520d36d72a75eab525, 4000a14f1fc...",382,382,0.159359,113.98,perfumery,2.0,1.0,credit_card,NaT,NaN
683cda1913456fb0cca302892b37a052,2,"[2f5124f516a7f5e4095ef4325d67c511, 304137040ea...",301,285,0.202243,129.90,watches_gifts,2.0,1.0,cash,NaT,NaN
7d08a8005756c215935d45bcc0e9a760,2,"[46936461f0c4e3c80b9289ce5fc1682a, 19541624666...",646,646,0.094234,259.80,furniture_decor,2.0,1.0,credit_card,NaT,NaN
8b1aed2dad15fcad8eb2a5c555e2d26e,2,"[979440acde78c7c04e8c8a92f2bc8ada, 516ddf29a9e...",572,572,0.106425,74.80,fashion_bags_accessories,2.0,5.0,voucher,NaT,NaN


In [66]:
data.average_delivery_time.isnull().value_counts()

False    2979
True       18
Name: average_delivery_time, dtype: int64

In [67]:
data.delay_rate.isnull().value_counts()

False    2979
True       18
Name: delay_rate, dtype: int64

In [57]:
data.sample(1).orders_ids.iloc[0]

array(['8ac915a1f508baa87e8fc8d596aa6fc1',
       '9eefad52e5b37dc3e94fc823d63057b5'], dtype=object)

In [58]:
len(order_items.order_id.unique())

AttributeError: 'DataFrame' object has no attribute 'order_id'

In [ ]:
order_items.order_item_id.value_counts()

Seulement 9803 commandes ont au moins deux items